In [6]:
# installs and memory mgmt
%pip install transformers datasets accelerate peft
%pip install scikit-learn
%pip install auto-gptq
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
!pip install -U bitsandbytes
from transformers import AutoModelForCausalLM
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import pipeline
import torch
torch.cuda.empty_cache()

import gc
gc.collect()


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


2067

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

torch.cuda.empty_cache()  # Clear GPU cache

pipe = pipeline(
    "text-generation",
    model="meta-llama/Llama-3.2-1B-Instruct",
    device=0,  # Set to GPU
    torch_dtype=torch.float16,
    max_length=128  # Reduce output length for lower memory usage
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B", device_map="auto")

messages = [
    {"role": "user", "content": "Who are you?"},
]

output = pipe(
    messages,
    max_new_tokens=50,
    pad_token_id=None
)

print(output)

Device set to use cuda:0


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [10]:
df1 = pd.read_json("merged_cwl_documents.json")
df2 = pd.read_json("merged_cwl_documents.json")

df = pd.concat([df1, df2])
df.columns

Index(['_id', 'description', 'source', 'content', 'embedding', 'class',
       'inputs', 'outputs', 'baseCommand', 'hints', 'cwlVersion', 'arguments',
       'requirements', 'createdAt', 'updatedAt'],
      dtype='object')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

# the columns we want to use are in the target
target_column = ['description', 'inputs', 'outputs', 'content']

# combine all target columns into a single field
df["combined"] = df.apply(
    lambda row: " ".join(str(row[col]) for col in target_column if col in df.columns and pd.notnull(row[col])),
    axis=1
)
df = df[df["combined"].notnull()]

# traintest split 
train_df, val_df = train_test_split(df, test_size=0.2)
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)



from transformers import AutoTokenizer

# we need to tokenize the data 
tokenizer = AutoTokenizer.from_pretrained("Llama-3.2-1B-Instruct")

# padding token 
if tokenizer.pad_token is None:
    if tokenizer.eos_token:
        tokenizer.pad_token = tokenizer.eos_token  # Use eos_token as pad_token
    else:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Add a new pad_token
        model.resize_token_embeddings(len(tokenizer))  # Resize model embeddings if necessary
        
# Define preprocessing function
def preprocess_function(examples):
    return tokenizer(examples["combined"], truncation=True, padding="max_length")


train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/1256 [00:00<?, ? examples/s]

In [ ]:
# memory mgmt
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.allow_tf32 = False
